# ***Image generation, automatic mask generation and inpainting starting with Stable Diffusion XL 1.0 and Segment Anything Model - Then share it with your colleagues on the Internet and collaborate with them!***
![Generated image, generated mask and inpainted image](cat_in_sailor_suit.png)

# **Prerequisites**
**Amazon SageMaker Studio**

- This notebook is designed to run in the Amazon SageMaker Studio environment.

**Quota:**

- ml.g5.2xlarge for endpoint usage >= 1

**Endpoint:**

- You have the option to use an existing endpoint, or you can deploy an endpoint within this notebook. To deploy a new endpoint, you first need to subscribe to the SDXL Model Package.

    To subscribe to the SDXL Model Package:
    
    1.Open the SDXL Model Package listing page: https://aws.amazon.com/marketplace/pp/prodview-pe7wqwehghdtm
    
    2.On the AWS Marketplace listing, click on the Continue to subscribe button.
    
    3.On the Subscribe to this software page, review and click on "Accept Offer" if you and your organization accept the EULA, pricing, and support terms.

**Notebook Environment:**

- **Image:**
    PyTorch 2.0.0 GPU optimized Python 3.10

- **Kernel:**
    Python3

- **Instance Type:**
    GPU instances such as ml.g4dn.xlarge (RECOMMEND)

> This notebook can be run on an instance without a GPU, but it takes a very long time to generate masks, so I strongly recommend using a GPU instance. If you're going to use an instance type without a GPU, you'll need an instance type with at least 8GB of memory. If the memory is insufficient, the kernel will crash.
> Also, please note that the results of segmentation can vary depending on whether you use GPU or CPU.

# **Security Notes**

**In this notebook, we are using Gradio (https://www.gradio.app/) with its share option. The reason for using Gradio is that it allows us to easily display an image gallery on the notebook and share it with others over the internet. Without the 'share' option, Gradio will not work in the SageMaker Studio environment.　If you do not wish to share over the internet, consider implementing your own code for displaying the gallery of mask images and the final inpainted images.**

# **Execute this notebook**

**If you run "Run All Cells," this notebook will stop at the gallery display of the generated mask images. Select the mask from the gallery that matches the area to be inpainted, and press the Enter key in the input area directly below the paused cell to resume the notebook's execution. Let's run it line by line at first to see what this notebook is doing.**

# **Checking Prerequisites**

In [ ]:
try:
    import torch
    print("PyTorch is installed. Please run next cell.")
except ImportError:
    print("PyTorch is not installed. Please check your docker container image. Your should use PyTorch 2.0.0 Python 3.10.")

In [ ]:
if torch.cuda.is_available():
    print("PyTorch can use GPU. Please run next cell.")
else:
    print("PyTorch cannot use GPU. Please check your instance type. GPU instance is better.")

# **Installing libraries**

In [ ]:
### General Libraries
!pip install -q --upgrade pip
!pip install -q opencv-python
!pip install -q gradio
!pip install -q ipywidgets 
### stability.ai SDK Library
!pip install -q 'stability-sdk[sagemaker] @ git+https://github.com/Stability-AI/stability-sdk.git@sagemaker'
### You can ignore all WARNIG messages from pip.

In [ ]:
### Meta AI's Segment Anything Model(SAM) libraries
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!pip install -q pycocotools matplotlib onnxruntime onnx
### You can ignore all WARNIG messages from pip.

In [ ]:
### Import libraries
import os
import io

import boto3
import sagemaker
from sagemaker import ModelPackage, get_execution_role

from stability_sdk_sagemaker.predictor import StabilityPredictor
from stability_sdk.api import GenerationRequest, GenerationResponse, TextPrompt

import PIL, cv2
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64, json
from matplotlib import pyplot as plt
import numpy as np
from numpy import asarray

import gradio as gra

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

 # **Choose existing endpoint or create new one for Stable Diffusion XL**

In [ ]:
### List your existing endpoint
!aws sagemaker list-endpoints

In [ ]:
### If you would like to use existing endpoint, set your endpoint name in the code.
### If you are creating a new endpoint, please leave it blank.
endpoint_name = ""

In [ ]:
if endpoint_name == "":
    new_endpoint = True
    ### Choose your endpoint name
    from sagemaker.utils import name_from_base
    endpoint_name=name_from_base('sdxl-1-0-jumpstart') # change this as desired
    print(f"Creating new endpoint {endpoint_name}...")
else:
    new_endpoint = False
    print(f"Using existing endpoint {endpoint_name}.")

**Get the Model Package ARN using the map below:**

In [ ]:
sagemaker_session = sagemaker.Session()


**Create endpoint**

In [ ]:
if new_endpoint == True:
    model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6"
    }


    region = boto3.Session().region_name
    if region not in model_package_map.keys():
        raise ("UNSUPPORTED REGION")
    package_arn = model_package_map[region]  # TODO

    role_arn = get_execution_role()

    model = ModelPackage(role=role_arn,model_package_arn=package_arn,sagemaker_session=sagemaker_session,predictor_cls=StabilityPredictor)
    # Deploy the ModelPackage. This will take 5-10 minutes to run
    instance_type="ml.g5.2xlarge" # valid instance types for this model are ml.g5.2xlarge, p4d.24xlarge, and p4de.24xlarge
    deployed_model = model.deploy(initial_instance_count=1,instance_type=instance_type,endpoint_name=endpoint_name)

# **Prepare to connect to endpoint**

In [ ]:
### List your endpoint
!aws sagemaker list-endpoints

In [ ]:
deployed_model = StabilityPredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

# **Text to image**
**The notebook comes pre-populated with prompts to generate three objects: "PERSON", "CAR", and "CAT". Please choose which object you would like to generate.**

**Of course you can tweak the prompts to create your own wonderful creations. Once you've tried each of the prompts provided,let's tinker with the prompts!**

In [ ]:
### Uncomment the line of the object you wish to generate.
#object = "PERSON"
#object = "CAR"
object = "CAT"

In [ ]:
### Prepare t2i prompts
if object == "PERSON":
    positive_prompt = """Photo realistic, detailed 8K, Beautiful girl in camisole dress standing on sandy beach 
    with sea in background,  long blonde hair, blue sky ,closeup character portrait, cute detailed digital art,
    japanese anime, 1girl"""
    negative_prompt = "ugly, deformed"
elif object == "CAR":
    positive_prompt = """Photo realistic, detailed 8K, Sports car parked on the beach with the sea in the background"""
    negative_prompt = "ugly, deformed"
elif object == "CAT":
    positive_prompt = """Photo realistic, detailed 8K, Clothed cat sitting on a chair"""
    negative_prompt = "ugly, deformed"
else:
    print("Please check drawing object.")

In [ ]:
print(f"positive_prompt = {positive_prompt}")
print(f"negative_prompt = {negative_prompt}")

In [ ]:
### Call Stable Diffusion Model
output = deployed_model.predict(GenerationRequest(text_prompts=[TextPrompt(text=positive_prompt,weight=1), 
                                                                TextPrompt(text=negative_prompt,weight=-1)],
                                                  style_preset="cinematic",
                                                  #sampler = "K_DPMPP_2S_ANCESTRAL",
                                                  sampler="K_EULER_ANCESTRAL",
                                                  steps= 50,
                                                  cfg_scale=5,
                                                  samples=1,
                                                  seed = 2574847677, #3995923165,
                                                  width=1024,
                                                  height=1024.
                                                 ))

In [ ]:
### Helper function to display generated image
def decode_and_show(model_response: GenerationResponse) :
    """
    Decodes and displays an image from SDXL output

    Args:
        model_response (GenerationResponse): The response object from the deployed SDXL model.

    Returns:
        PNG Image
    """
    image = model_response.artifacts[0].base64
    image_data = base64.b64decode(image.encode())
    image = Image.open(io.BytesIO(image_data))
    print(image.format)
    display(image)
    return(image)

source_image = decode_and_show(output)
source_image.save(f"{object}-generated.png")

# **Generate Masks**

**Setup Meta AI's Segment Anything Model(SAM) to automatically generate masks**

For more information on the Segment Anything Model, please refer to the following link.

https://github.com/facebookresearch/segment-anything/

In [ ]:
### Downloading Segment Anything Model Checkpoint file
checkpoint_file_name = "sam_vit_h_4b8939.pth"
if not os.path.exists(checkpoint_file_name):
    print(f"Download Segment Anything Checkpoint {checkpoint_file_name}")
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
else:
    print(f"Segment Anything Checkpoint {checkpoint_file_name} already exists.")

In [ ]:
### convert the image into an array in order to use it later with the segmentation model
segmentation_image = asarray(source_image)

In [ ]:
### Initialize Segment Anything Model

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

### If CUDA is not available in thid environment, cpu will be used.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.99, # A filtering threshold in [0,1], using the model's predicted mask quality. ex.0.99
    stability_score_thresh=0.90, #0.92
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)


In [ ]:
### Run Segment Anything Model
masks = mask_generator.generate(segmentation_image)

### Print number of masks generated and the keys attached to each of them
print(f"Number of masks generated: {len(masks)}")
print(masks[0].keys())

In [ ]:
###　Helper function to display color-coded masks generated over the original imag
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(segmentation_image)
show_anns(masks)
plt.axis('off')
plt.show() 

# **Display all masks in gallery format**

*Select the mask image you wish to use for inpainting by clicking on it from the gallery displayed below. Once you have selected on a mask, move the mouse cursor to the next cell and run the cell(Shift-clicking does not automatically move the cursor).*

In [ ]:
### Helper function to display all masks in gallery format
mask_images = []

for i in range(len(masks)):
    segmentation_mask=masks[i]['segmentation']
    mask_images.append((PIL.Image.fromarray(segmentation_mask), str(i)))

def process_image(selected_image: gra.SelectData):
    global mask_index
    mask_index = selected_image.index
    

with gra.Blocks() as allmasks:
    gra.Gallery(mask_images,columns=[4], rows=[len(masks)/4+1], object_fit="contain", height="auto").select(process_image)
    
allmasks.launch(share=True)

**If you share the public URL above with your colleagues, they too can view this gallery of mask images and select a mask.**

**Confirm mask id you want to use for inpainting**

*If your execution stops at the cell immediately below, it means that the notebook is waiting for you to select a mask image. Select the mask image from the gallery above and press enter in the area directly below the next cell that is waiting for input. The execution will then resume and the inpainting will be performed using the mask image you selected.*

In [ ]:
# Display mask id you selected.
while 'mask_index' not in globals():
    input("Please select a mask from the gallery above. Once you have chosen, press enter here to continue.")

print(f"Mask ID you selected is {mask_index}")

In [ ]:
segmentation_mask=masks[mask_index]['segmentation']
mask_image=PIL.Image.fromarray(segmentation_mask)
display(mask_image)
mask_image.save(f"{object}-mask-{mask_index}.png")

# **Inpaint**

In [ ]:
### If you prefer to use a pre-prepared mask, you can use the code below. 
### In that case, remove the triple quotation marks below.
""" 
rgba_image = Image.open('CAT-mask-7-edited.png')
if rgba_image is None:
    print('Failed to load image. Please check the file name and path.')
else:
    print('Mask image successfully loaded.')
    print(rgba_image.format)
mask_image = rgba_image.convert('RGB')
print(mask_image.format)
"""

In [ ]:
### Setup Stable Diffusion In-Paint prompts
if object == "PERSON":
    positive_prompt = """Photo realistic, detailed 8K, Cherry blossom petal pattern dress"""
    negative_prompt = "ugly, deformed"
elif object == "CAR":
    positive_prompt = """Photo realistic, detailed 8K, yellow Sports car"""
    negative_prompt = "ugly, deformed"    
else:
    positive_prompt = """Photo realistic, detailed 8K, Cat in sailor suit"""
    negative_prompt = "ugly, deformed"

In [ ]:
print(f"positive_prompt = {positive_prompt}")
print(f"negative_prompt = {negative_prompt}")

In [ ]:
### Helper function: Encode the image in Base64
def encode_img(img):
    encoded_img = base64.b64encode(img).decode()
    return encoded_img

### Helper function: Convert PNG image to JPEG image
def convert_image_to_jpeg(image):
    bytestream = io.BytesIO()
    image.save(bytestream, format='JPEG')
    byte_data = bytestream.getvalue()
    return byte_data

### Call Stable Diffusion XL 1.0 Model
inpaint_output = deployed_model.predict(GenerationRequest(text_prompts=[TextPrompt(text=positive_prompt,weight=1),
                                                                TextPrompt(text=negative_prompt,weight=-1)],
                                                  init_image = encode_img(convert_image_to_jpeg(source_image)),
                                                  mask_source = "MASK_IMAGE_WHITE",
                                                  mask_image = encode_img(convert_image_to_jpeg(mask_image)),
                                                  cfg_scale =9,
                                                  clip_guidance_preset = "NONE",
                                                  sampler = "K_EULER_ANCESTRAL",
                                                  samples = 1,
                                                  seed = 44332211, #3 (for CAT)
                                                  steps = 50,
                                                  style_preset = "cinematic"
                                                 ))

### Display the inpainted image
inpainted_image = decode_and_show(inpaint_output)

In [ ]:
inpainted_image.save(f"{object}-inpainted.png")

In [ ]:
with gra.Blocks() as results:
    gra.Gallery([(source_image,"Initially generated image"),(mask_image,"Used mask"),(inpainted_image,"Inpainted image")],columns=[3], rows=[1], object_fit="contain", height="auto")
    #gra.Gallery([source_image,inpainted_image,mask_image]).style(columns=[3], rows=[1], object_fit="contain", height="auto")
results.launch(share=True)

**Share the public URL above and your friends will be able to view your amazing creations.**

# **Delete Endpoint**

In [ ]:
!aws sagemaker list-endpoints

**Endpoints can be recreated at any time using this notebook or "Endpoint configuration". When an endpoint is not in use for a long period of time, it should be deleted. To delete an endpoint, comment out the code below (deployed_model.sagemaker_session.delete_endpoint(endpoint_name)) and execute it.**

In [ ]:
deployed_model.sagemaker_session.delete_endpoint(endpoint_name)

**To confirm your endpoint gone, run next cell.**

In [ ]:
import time
time.sleep(10)
!aws sagemaker list-endpoints